## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("../")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,37.06,87,75,4.61,broken clouds
1,1,Felanitx,ES,39.4696,3.1483,77.13,86,16,7.36,few clouds
2,2,Hasaki,JP,35.7333,140.8333,90.28,75,22,12.15,few clouds
3,3,Sitka,US,57.0531,-135.3300,57.09,82,75,12.66,light rain
4,4,Touros,BR,-5.1989,-35.4608,74.30,80,0,15.28,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Felanitx,ES,39.4696,3.1483,77.13,86,16,7.36,few clouds
8,8,Xinxiang,CN,35.3089,113.8672,85.46,66,2,1.99,clear sky
12,12,Butaritari,KI,3.0707,172.7902,83.95,72,13,12.24,light rain
14,14,Atuona,PF,-9.8000,-139.0333,78.39,70,16,17.27,few clouds
17,17,Bilma,NE,18.6853,12.9164,88.36,43,94,5.93,overcast clouds
18,18,Kapaa,US,22.0752,-159.3190,85.98,79,90,4.00,light rain
23,23,Hambantota,LK,6.1241,81.1185,79.14,100,75,11.50,broken clouds
25,25,Gat,IL,31.6100,34.7642,77.92,100,15,0.47,few clouds
30,30,Hilo,US,19.7297,-155.0900,80.33,85,90,8.05,overcast clouds
31,31,Puerto Escondido,MX,15.8500,-97.0667,82.69,83,100,6.46,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                278
City                   278
Country                278
Lat                    278
Lng                    278
Max Temp               278
Humidity               278
Cloudiness             278
Wind Speed             278
Current Description    278
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                278
City                   278
Country                278
Lat                    278
Lng                    278
Max Temp               278
Humidity               278
Cloudiness             278
Wind Speed             278
Current Description    278
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Felanitx,ES,77.13,few clouds,39.4696,3.1483,
8,Xinxiang,CN,85.46,clear sky,35.3089,113.8672,
12,Butaritari,KI,83.95,light rain,3.0707,172.7902,
14,Atuona,PF,78.39,few clouds,-9.8000,-139.0333,
17,Bilma,NE,88.36,overcast clouds,18.6853,12.9164,
18,Kapaa,US,85.98,light rain,22.0752,-159.3190,
23,Hambantota,LK,79.14,broken clouds,6.1241,81.1185,
25,Gat,IL,77.92,few clouds,31.6100,34.7642,
30,Hilo,US,80.33,overcast clouds,19.7297,-155.0900,
31,Puerto Escondido,MX,82.69,overcast clouds,15.8500,-97.0667,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Felanitx,ES,77.13,few clouds,39.4696,3.1483,Petit Hotel Hostatgeria Sant Salvador
8,Xinxiang,CN,85.46,clear sky,35.3089,113.8672,Guoyu Holiday Inn
12,Butaritari,KI,83.95,light rain,3.0707,172.7902,Isles Sunset Lodge
14,Atuona,PF,78.39,few clouds,-9.8000,-139.0333,Villa Enata
17,Bilma,NE,88.36,overcast clouds,18.6853,12.9164,Expeditions Ténére voyages


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'clean_hotel_df' is not defined